In [1]:
# Class Vertice
class Vertex:
    '''Estrutura de Nó para um grafo: um elemento que é o identificador deste nó'''

    def __init__(self, x):
        '''O vértice será inserido no Grafo usando o método insert_vertex(x) que cria um Vertex'''
        self._elemento = x

    def __hash__(self):
        ''' este vértice (o seu identificador) é usado como chave'''
        return hash(id(self))  # devolve um inteiro que identifica este vértice como uma chave num dicionário

    def __str__(self):
        return'{0}'.format(self._elemento)

    def __eq__(self, x):
        return x == self._elemento

    def vertice(self):
        '''Devolve o nome deste vértice'''
        return self._elemento


# #### Class Edge
class Edge:
    '''Estrutura de Aresta para um Grafo: (origem, destino) e peso '''

    def __init__(self, u, v, p):
        self._ant = u
        self._suc = v
        self._weight = p

    def __hash__(self):
        # para associar a aresta a uma chave para um dicionário
        return hash( (self._ant, self._suc) )

    def __str__(self):
        '''Mostra a Aresta para um Grafo: (origem, destino) - peso '''
        return'({0},{1})-{2} '.format(self._ant, self._suc, self._weight)

    def __eq__(self, other):
        # define igualdade de duas arestas
        return self._ant == other._ant and self._suc == other._suc

    def endpoints(self):
        '''devolve (u,v) para indicar os vértices antecessor e sucessor.'''
        return (self._ant, self._suc)

    def opposite(self, v):
        '''Indica o vértice oposto ao v neste arco; v tem de ser um dos vértices.'''
        return self._suc if v is self._ant else self._ant

    def cost(self):
        '''Devolve o peso associado a este arco.'''
        return self._weight

    def show_edge(self):
        print('(',self._ant, ', ', self._suc, ') com peso', self._weight)

In [2]:
class Graph(Vertex, Edge):
    '''Representação de um grafo usando mapeamentos de adjacências (associações) - dictionaries'''

    def __init__(self, directed=False):
        '''Cria um grafo vazio (dicionário de _vertices); é orientado se o parâmetro directed tiver o valor True'''
        self._directed = directed
        self._n = 0                 # quantidade de nós no Grafo
        self._m = 0                 # quantidade de arcos no Grafo
        self._vertices = {}         # dicionário com chave vértice e valor dicionário de adjacências

    def insert_vertex(self, x):
        '''Insere e devolve um novo vértice com o elemento x'''
        v = Vertex(x)
        self._vertices[v] = {}      # inicializa o dicionário de adjacências deste vértice a vazio
        self._n +=1                 # mais um vértice no grafo
        return v

    def insert_edge(self, u, v, x):
        '''Cria e insere uma nova aresta entre u e v com peso x'''
        e = Edge(u, v, x)
        self._vertices[u][v] = e  # vai colocar nas adjacências de u
        self._vertices[v][u] = e  # e nas adjacências de v (para facilitar a procura de todos os arcos incidentes em ou originários de)
        self._m +=1

    def is_directed(self):
        '''com base na criação original da instância, devolve True se o Grafo é dirigido; False senão '''
        return self._directed  # True se os dois contentores são distintos

    def order(self):
        '''Ordem de um grafo: a quantidade de vértices no Grafo'''
        return self._n

    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo'''
        return self._vertices.keys()

    def size(self):
        '''Dimensão de um grafo: a quantidade total de arestas do Grafo'''
        return self._m

    def set_of_edges(self):
        '''Devolve o conjunto (set) de todas as arestas do Grafo'''
        result = set()      # avoid double-reporting edges in undirected graph
        for secondary_map in self._vertices.values():
            result.update(secondary_map.values())  # add edges to resulting set
        return result


    def degree(self, v, outgoing=True):
        '''Quantidade de arestas originárias ou incidentes no vértice v
           Se for um grafo dirigido, conta as arestas outgoing ou incoming,
           de acordo com o valor de outgoing (True or False)
        '''
        adj = self._vertices
        if not self._directed:
            count = len(adj[v])
        else:
            count = 0
            for edge in adj[v].values():
                x, y = edge.endpoints()
                if (outgoing and x == v) or (not outgoing and y == v):
                    count += 1
        return count


    def get_edge(self, u, v):
        '''Método interno: Devolve a aresta que liga u a v ou None se não forem adjacentes'''
        edge = self._vertices[u].get(v)     # returns None se não existir aresta entre u e v
        if edge and self._directed: # se é dirigido
            x = edge.endpoints()        # vai confirmar se é o arco u --> v
            if x[1] != v:
                edge = None
        return edge

    def remove_edge(self, u, v):
        '''Remove a aresta entre u e v '''
        if  u in self._vertices.keys() and v in self._vertices[u].keys():
            del self._vertices[u][v]
            del self._vertices[v][u]
            self._m -=1

    def remove_vertex(self, v):
        '''remove o vértice v'''
        # remover todas as arestas de [v]
        # remover todas as arestas com v noutros vertices
        # remover o vértice v
        if v in self._vertices.keys():
            lst = [i for i in self.incident_edges(v)]
            for i in lst:
                x, y = i.endpoints()
                self.remove_edge(x,y)
            del self._vertices[v]
            self._n -=1
        #return v


    #outros métodos auxiliares#
    def incident_edges(self, v, incoming=True):
        '''Gerador: indica todas as arestas incoming de v
           Se for um grafo dirigido e incoming for False, devolve as arestas outgoing
        '''
        for edge in self._vertices[v].values(): # para todas as arestas relativas a v:
            if not self._directed:
                    yield edge
            else:  # senão deve ir procurar em todas as arestas para saber quais entram ou saiem
                x, y = edge.endpoints()
                if (incoming and y == v) or (not incoming and x == v):
                    yield edge

    def printG(self):
        '''Mostra o grafo por linhas'''
        if self._n == 0:
            print('O grafo está vazio!')
        else:
            print('Grafo orientado:', self._directed)
            for v in self.vertices():
                print('\nvertex ', v, ' grau_in: ', self.degree(v,False), end=' ')# mostra o grau (de entrada, se orientado)
                for i in self.incident_edges(v):
                    print(' ', i, end=' ')
                if self._directed:          # se orientado, mostrar o grau de saída
                    print('\n \t   grau_out: ', self.degree(v, True), end=' ')
                    for i in self.incident_edges(v, False):    # e mostra os arcos de saída
                        print(' ', i, end=' ')

In [4]:
import pandas as pd

# Ler os dados do arquivo Excel
df = pd.read_excel("London_metro_Interstation_v3.xlsx")

# Criar um dicionário de listas vazio para representar o grafo
graph = Graph()

# Iterar sobre cada linha do DataFrame e adicionar as arestas ao grafo
for index, row in df.iterrows():
    source = row['From Station Id']
    destination = row['To Station Id']
    weight = row['Distance (Kms)']

    if source not in graph:
        graph[source] = []

    if destination not in graph:
        graph[destination] = []

 # Adicionar aresta nas duas direções, já que o grafo é não-direcionado
graph[source].append((destination, weight))
graph[destination].append((source, weight))
graph.printG()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

Questão 1.

Notas: usar heap binary????
Min heap-> Uma heap binária é uma estrutura de dados que organiza os elementos de forma que o elemento de maior prioridade sempre esteja no topo da heap. No caso de uma min heap (heap mínima), o elemento de menor valor é colocado no topo.

A estrutura de heap binária é especialmente útil na implementação de algoritmos de prioridade, como o algoritmo de Dijkstra, pois permite acesso rápido ao elemento de maior prioridade e também suporta operações eficientes de inserção e remoção.

O algoritmo de Dijkstra utiliza uma min heap para manter os nós não visitados, priorizando o nó com a menor distância até o momento. Isso permite que o algoritmo selecione eficientemente o nó de menor distância em cada iteração, garantindo um desempenho eficiente em termos de tempo de execução.
https://www.geeksforgeeks.org/min-heap-in-python/
--------------------------------------------------------------------------------------------------
A escolha entre uma heap binária e uma Fibonacci heap depende do contexto e dos requisitos específicos do seu problema.

Heap binária:

Vantagens:
Implementação relativamente simples e direta.
Baixo consumo de memória.
Bons tempos de execução para operações de inserção, remoção e acesso ao elemento de menor prioridade (topo da heap).
Desvantagens:
O tempo de execução do algoritmo de Dijkstra usando uma heap binária é O((V + E) log V), onde V é o número de vértices e E é o número de arestas. Essa é a complexidade de tempo média para uma operação de relaxamento de aresta.
As operações de diminuição de chave (quando uma distância é atualizada) podem ter um tempo de execução relativamente alto, chegando a O(log V).
Fibonacci heap:

Vantagens:
O tempo de execução do algoritmo de Dijkstra usando uma Fibonacci heap é O(V log V + E), o que é assintoticamente mais rápido do que uma heap binária.
Melhor desempenho em cenários com muitas operações de diminuição de chave.
Desvantagens:
Implementação mais complexa e requer mais memória em comparação com a heap binária.
Pode ter um tempo de execução mais lento para operações de inserção e remoção em comparação com a heap binária.
Em geral, se você está lidando com problemas menores ou com requisitos de desempenho mais modestos, uma heap binária é uma escolha sólida devido à sua simplicidade e baixo consumo de memória. No entanto, se você está trabalhando com problemas maiores, em que o desempenho é crítico e há muitas operações de diminuição de chave, uma Fibonacci heap pode ser uma opção mais eficiente.

É importante ressaltar que, embora a Fibonacci heap possa ter um desempenho teoricamente superior em termos de complexidade assintótica, em situações práticas, a diferença real de desempenho pode não ser tão significativa. Portanto, é recomendado fazer uma análise detalhada do contexto e realizar testes para determinar qual estrutura de dados é mais adequada para o seu caso específico.